# setting

In [3]:
# Load library
import mne, os, pickle, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Qt5Agg')

# concatenate EEG data

In [2]:
# # define data directory
# subj = 'sub-004'
# data_type = 'EEG'
# data_dir  = os.path.realpath('../dataset/%s/%s/%s')%(data_type, subj, data_type.lower())

# # load EEG data
# sessions = ['sess-oddball01', 'sess-oddball02', 'sess-oddball03']
# raws = []

# for s, sess in enumerate(sessions):
#     sess_fname = os.path.join(data_dir, '%s_%s_ICA-raw.fif') %(subj, sess)
#     raw   = mne.io.read_raw(sess_fname, preload=True)
#     raws.append(raw)

# # concatenate raw data from different sessions
# data = mne.concatenate_raws(raws)

# # save file
# fname = os.path.join(data_dir, '%s_sess-oddball_ICA-raw.fif') %(subj)
# data.save(fname, overwrite=True)

# load EEG data

In [4]:
# define data directory
subj = 'sub-004'
data_type = 'EEG'
data_dir  = os.path.realpath('../dataset/%s/%s/%s')%(data_type, subj, data_type.lower())

# load EEG data
fname = os.path.join(data_dir, '%s_sess-oddball_ICA-raw.fif') %(subj)
data  = mne.io.read_raw(fname, preload=True)

Opening raw data file /Users/cl5564/Library/CloudStorage/Dropbox/NYUC&P/Class/Year5-1_2024Fall/MEEG_methods/Lab/dataset/EEG/sub-004/eeg/sub-004_sess-oddball_ICA-raw.fif...
    Range : 97280 ... 1587202 =    190.000 ...  3100.004 secs
Ready.
Reading 0 ... 1489922  =      0.000 ...  2910.004 secs...


# read experiment log file
original event trigger codes
- 1: classicalAud
- 2: semanticVis
- 4: semanticAud
- 64: even
- 128: odd

In [4]:
# read log file
log_fname = os.path.join(data_dir, '%s_task-oddball_events.tsv') %(subj)
log_df = pd.read_csv(log_fname, sep='\t')

# look for events in the oddball sessions (exclude the story sessions)
ob_idx = log_df['blockTypeActual'].isin(['classicalAud', 'semanticAud', 'semanticVis'])
ob_df  = log_df[ob_idx]
ob_df  = ob_df.dropna()
print(ob_df.shape)

# define oddball types
oddball_conds = {'classicalAud': 1, 'semanticVis': 2, 'semanticAud': 4}

# create new condition codes
cond_codes = np.zeros(ob_df.shape[0], dtype=int)

for cond, num in oddball_conds.items():
    cond_idx = ob_df['blockTypeActual'].isin([cond])
    cond_codes[cond_idx] = num

cond_codes = cond_codes*1000 + ob_df['trigger_codes_fixed'].values

# add new condition codes to the dataframe
ob_df['cond_codes'] = cond_codes


(1440, 14)


# update event code


In [5]:
# find events
events = mne.find_events(data, min_duration=0.002)

# look for trial indices
trl_idx = (events[:,2] == 64) | (events[:,2] == 128)

# update event code
events[trl_idx,2] = cond_codes

1447 events found
Event IDs: [  1   2   4  64 128]


# epoch segment

In [14]:
# apply 15 Hz low-pass filter
data_filt = data.copy()
data.filter(l_freq=None, h_freq=15, method='fir')

# define epoch parameters
epoch_tmin     = -1
epoch_tmax     = 2
epoch_baseline = (-0.5, 0)
detrend        = 1 # linear detrend
decim          = 1 # 1 for no decimation (downsampling)

# define event id
event_id = dict(classicalAud_even = 1064, classicalAud_odd = 1128,
                semanticVis_even  = 2064, semanticVis_odd  = 2128,
                semanticAud_even  = 4064, semanticAud_odd  = 4128)

# epoching
info   = data.info
picks  = mne.pick_types(info, meg=False, eeg=True, eog=False, stim=False)
epochs = mne.Epochs(data,
                    events   = events, 
                    event_id = event_id,
                    picks    = picks,
                    decim    = decim,
                    tmin     = epoch_tmin,
                    tmax     = epoch_tmax,
                    baseline = epoch_baseline,
                    detrend  = detrend,
                    preload  = True,
                    reject_by_annotation = False)


Filtering raw data in 3 contiguous segments
Setting up low-pass filter at 15 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 15.00 Hz
- Upper transition bandwidth: 3.75 Hz (-6 dB cutoff frequency: 16.88 Hz)
- Filter length: 451 samples (0.881 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 256 out of 256 | elapsed:    7.1s finished


Not setting metadata
1440 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1440 events and 1537 original time points ...
0 bad epochs dropped


# manually reject bad epochs

In [17]:
epochs_cleaned = epochs.copy()
epochs_cleaned.plot(n_epochs=5, n_channels=50, events=events,
                    butterfly=True, scalings=dict(eeg=8e-4))


Using pyopengl with version 3.1.6


Dropped 231 epochs: 0, 3, 4, 7, 8, 9, 13, 14, 28, 40, 41, 50, 52, 53, 54, 57, 61, 62, 79, 80, 86, 87, 91, 92, 109, 110, 111, 120, 136, 137, 138, 139, 152, 153, 154, 159, 160, 171, 172, 209, 223, 253, 254, 265, 266, 273, 300, 328, 329, 335, 336, 340, 341, 343, 344, 354, 360, 361, 366, 390, 391, 407, 408, 414, 438, 439, 442, 443, 448, 449, 452, 453, 455, 456, 461, 462, 463, 464, 466, 467, 472, 473, 474, 475, 476, 479, 481, 492, 510, 511, 519, 520, 530, 531, 532, 540, 541, 542, 570, 573, 579, 583, 585, 586, 595, 606, 614, 615, 617, 629, 650, 651, 680, 681, 704, 713, 714, 715, 716, 717, 720, 721, 750, 758, 761, 762, 800, 809, 810, 812, 813, 865, 869, 886, 887, 888, 893, 894, 895, 900, 901, 904, 905, 907, 908, 922, 925, 926, 930, 938, 939, 944, 945, 953, 954, 960, 961, 974, 975, 976, 981, 982, 983, 1020, 1024, 1046, 1047, 1048, 1049, 1050, 1072, 1073, 1087, 1088, 1105, 1107, 1110, 1111, 1114, 1127, 1140, 1146, 1148, 1149, 1151, 1152, 1170, 1171, 1176, 1183, 1216, 1223, 1224, 1230, 1231, 123

In [7]:
# save epochs
epochs_fname = os.path.join(data_dir, '%s_sess-oddball_cleaned-epo.fif') %(subj)
epochs_cleaned.save(epochs_fname, overwrite=True)

# # read epochs
# epochs_fname = os.path.join(data_dir, '%s_sess-oddball_cleaned-epo.fif') %(subj)
# epochs_cleaned = mne.read_epochs(epochs_fname, preload=True)

Reading /Users/cl5564/Library/CloudStorage/Dropbox/NYUC&P/Class/Year5-1_2024Fall/MEEG_methods/Lab/dataset/EEG/sub-004/eeg/sub-004_sess-oddball_cleaned-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
1209 matching events found
No baseline correction applied
0 projection items activated


# plotting

### butterfly plot with topomaps

In [15]:
cond = 'semanticVis_even'
evk  = epochs_cleaned[cond].average()
evk.crop(tmin=-0.3, tmax=1)

evk.plot_joint(
              ts_args = dict(gfp=True, ylim=dict(eeg=[-8, 8]), time_unit='ms'),
              topomap_args = dict(vlim=[-6, 6], time_unit='ms')
              )

No projector specified for this dataset. Please consider the method self.add_proj.


<Figure size 1600x840 with 7 Axes>

### plot all channels

In [16]:
conds = ['classicalAud_even', 'classicalAud_odd']
evks = []
tmin, tmax = -0.3, 1

for c, cond in enumerate(conds):
    evk = epochs_cleaned[cond].average()
    evk.crop(tmin=tmin, tmax=tmax)
    evks.append(evk)

mne.viz.plot_evoked_topo(evks)


<Figure size 5120x2516 with 1 Axes>

### plot ERP from a subset of channels

In [ ]:
# crop epochs into smaller time winodw
tmin, tmax  = -0.3, 1
epochs_crop = epochs_cleaned.copy()
epochs_crop.crop(tmin, tmax)

In [ ]:
# split channels into anterior and posterior parts
layout = mne.channels.make_standard_montage('biosemi256')
chan_coor = np.array(list(layout.get_positions()['ch_pos'].values()))
chan_ant_idx = np.squeeze(np.where(chan_coor[:,1] >= 0))  # anterior channel indices
chan_pos_idx = np.squeeze(np.where(chan_coor[:,1] < 0))   # posterio channel indices

In [ ]:
# define conditions for comparision
conds = ['classicalAud_even', 'classicalAud_odd']

# figure prep
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(18,8))

# plot ERP averaged across the anterior channels
picks = np.array(epochs_crop.ch_names)[chan_ant_idx]
evokeds = dict(even = list(epochs_crop[conds[0]].iter_evoked()),
               odd  = list(epochs_crop[conds[1]].iter_evoked()))
p1 = mne.viz.plot_compare_evokeds(evokeds, picks=picks, combine="mean",
                                  ylim=dict(eeg=[-3, 3]), axes=ax1, show=False,
                                  title='Anterior region\n(%d sensors)'%(chan_ant_idx.size))

# plot ERP averaged across the posterior channels
picks = np.array(epochs_crop.ch_names)[chan_pos_idx]
evokeds = dict(even = list(epochs_crop[conds[0]].iter_evoked()),
               odd  = list(epochs_crop[conds[1]].iter_evoked()))
p2 = mne.viz.plot_compare_evokeds(evokeds, picks=picks, combine="mean", 
                                  ylim=dict(eeg=[-3, 3]), axes=ax2, show=False,
                                  title='Posterior region\n(%d sensors)'%(chan_pos_idx.size))

plt.rcParams['font.size'] = 20
plt.tight_layout()
plt.show()